# `method` on zf

# running `method` on zf

## imports

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install scanpy -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.0/129.0 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 kB 8.3 MB/s eta 0:00:00


In [3]:
!pip install -U plotly -q   # for making Sankey diagram

In [4]:
!pip install -U kaleido -q   # for saving Sankey diagram

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 27.8 MB/s eta 0:00:00


In [5]:
import pandas as pd
import scanpy as sc
import numpy as np
import torch
import sys
from scipy.spatial.distance import cdist
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import sys
import importlib
import plotly.graph_objects as go

In [6]:
filehandle_wdm = 'drive/Othercomputers/numac/GitHub/WDM/'
filehandle_save = 'drive/Othercomputers/numac/GitHub/celltypediscovery/_wdm_save_sc/'
filehandle_save_factors = 'drive/Othercomputers/numac/GitHub/celltypediscovery/_wdm_save_sc_factors/'
filehandle_load_moscot = 'drive/Othercomputers/numac/GitHub/celltypediscovery/_wdm_save_sc_moscot/'
filehandle_zf = 'drive/MyDrive/DX/_data/zebrafish/cleaned_common_pca_sc/'
filehandle_scaling = 'drive/Othercomputers/numac/GitHub/celltypediscovery/'

sys.path.insert(0, filehandle_wdm)
sys.path.insert(0, filehandle_save)
sys.path.insert(0, filehandle_zf)
sys.path.insert(0, filehandle_scaling)

import clustering
import util_LR
import util_zf
import FRLC_LRDist
import WassersteinDiffMap
import FRLC_LRDistScaling
import WassersteinDiffMapScaling

## reload

In [7]:
importlib.reload(clustering)
importlib.reload(util_LR)
importlib.reload(util_zf)
importlib.reload(FRLC_LRDist)

<module 'FRLC_LRDist' from '/content/drive/Othercomputers/numac/GitHub/WDM/FRLC_LRDist.py'>

## device

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'On device: {device}')
dtype = torch.float64

On device: cuda


## load zf: spatial, annotation hard clusterings as $Q$'s

In [9]:
# daniocell_dir = '/scratch/gpfs/ph3641/zebrafish/daniocell.h5ad'
# adata_daniocell = sc.read_h5ad(daniocell_dir)
# stages = np.unique(adata_daniocell.obs['hpf'].values)

zf_list = ['zf3', 'zf5', 'zf10', 'zf12', 'zf18', 'zf24']
zf_names = zf_list
filehandles_zf = [filehandle_zf + 'pair' + str(k) + '/' for k in range(len(zf_list))]
N = len(filehandles_zf)

spatial_list = []
exclude_rows = [None, None, None, None, None, None] # [1099, None, None, 325, None, None]

for i in range(len(zf_list)):
    file_spatial = filehandles_zf[i] + zf_names[i] +'_umap.npy'
    if i == len(zf_list) - 1:
        file_spatial = filehandles_zf[i-1] + zf_names[i] +'_umap.npy'
    spatial = np.load(file_spatial)
    nidx = exclude_rows[i]
    if nidx is not None:
        spatial = np.concatenate((spatial[:nidx,:], spatial[nidx+1:,:]))
    spatial_list.append(spatial)

In [10]:
'''
N = len(filehandles_zf)
Qs = [None]*(N-1)
Rs = [None]*(N-1)
adatas_Qs = [None] * (N-1)
adatas_Rs = [None] * (N-1)

key = 'celltype_new'

labels = []

for i in range(N-2, -1, -1):

    s1_name = zf_names[i]
    s2_name = zf_names[i+1]

    print(f'Slice pair {i}, aligning {s1_name} to {s2_name}')

    adata_pair = sc.read_h5ad(filehandle_zf + f'pair{i}/' + s1_name + '_' + s2_name + '.h5ad')

    adata1 = adata_pair[adata_pair.obs['timepoint'] == 1]
    # adata1 = adata1[adata1.obs['bin_annotation'] != 'Otic Vesicle']  # Exclude 'Otic Vesicle'

    # sc.pp.normalize_total(adata1)
    # sc.pp.log1p(adata1)

    adata2 = adata_pair[adata_pair.obs['timepoint'] == 2]
    # adata2 = adata2[adata2.obs['bin_annotation'] != 'Otic Vesicle']  # Exclude 'Otic Vesicle'
    # sc.pp.normalize_total(adata2)
    # sc.pp.log1p(adata2)
    adatas_Qs[i], adatas_Rs[i] = adata1, adata2

    encoder1 = OneHotEncoder(sparse_output=False)
    ys_onehot1 = encoder1.fit_transform(adata1.obs[key].values.reshape(-1, 1))
    _Q = ys_onehot1 / np.sum(ys_onehot1)

    encoder2 = OneHotEncoder(sparse_output=False)
    ys_onehot2 = encoder2.fit_transform(adata2.obs[key].values.reshape(-1, 1))
    _R = ys_onehot2 / np.sum(ys_onehot2)

    labels.append(list(encoder2.categories_[0]))

    if i == 0:
        labels.append(list(encoder1.categories_[0]))

    _Q, _R = torch.from_numpy(_Q).to(device).float(), torch.from_numpy(_R).to(device).float()

    # Filter rows
    nidx_1, nidx_2 = exclude_rows[i], exclude_rows[i+1]
    if nidx_1 is not None:
        _Q = torch.cat((_Q[:nidx_1,:], _Q[nidx_1+1:,:]))
    if nidx_2 is not None:
        _R = torch.cat((_R[:nidx_2,:], _R[nidx_2+1:,:]))

    Qs[i], Rs[i] = _Q, _R

Qs_gt_tensor = Qs + [Rs[4]]
Qs_gt_tensor = [ Q.type(torch.DoubleTensor).to(device) for Q in Qs_gt_tensor]

Qs_gt = [Q.cpu().numpy() for Q in Qs_gt_tensor]

adatas = adatas_Qs + [adatas_Rs[-1]]
'''
N = len(filehandles_zf)
Qs = [None]*(N-1)
Rs = [None]*(N-1)
adatas_Qs = [None] * (N-1)
adatas_Rs = [None] * (N-1)

key = 'celltype_new'

labels = []

for i in range(N-2, -1, -1):

    s1_name = zf_names[i]
    s2_name = zf_names[i+1]

    print(f'Slice pair {i}, aligning {s1_name} to {s2_name}')

    adata_pair = sc.read_h5ad(filehandle_zf + f'pair{i}/' + s1_name + '_' + s2_name + '.h5ad')

    adata1 = adata_pair[adata_pair.obs['timepoint'] == 1]
    adata2 = adata_pair[adata_pair.obs['timepoint'] == 2]
    adatas_Qs[i], adatas_Rs[i] = adata1, adata2

    encoder1 = OneHotEncoder(sparse_output=False)
    ys_onehot1 = encoder1.fit_transform(adata1.obs[key].values.reshape(-1, 1))
    _Q = ys_onehot1 / np.sum(ys_onehot1)

    encoder2 = OneHotEncoder(sparse_output=False)
    ys_onehot2 = encoder2.fit_transform(adata2.obs[key].values.reshape(-1, 1))
    _R = ys_onehot2 / np.sum(ys_onehot2)

    labels.append(list(encoder2.categories_[0]))

    if i == 0:
        labels.append(list(encoder1.categories_[0]))

    _Q, _R = torch.from_numpy(_Q).to(device).float(), torch.from_numpy(_R).to(device).float()

    # Filter rows
    nidx_1, nidx_2 = exclude_rows[i], exclude_rows[i+1]
    if nidx_1 is not None:
        _Q = torch.cat((_Q[:nidx_1,:], _Q[nidx_1+1:,:]))
    if nidx_2 is not None:
        _R = torch.cat((_R[:nidx_2,:], _R[nidx_2+1:,:]))

    Qs[i], Rs[i] = _Q, _R

Qs_gt_tensor = Qs + [Rs[4]]
Qs_gt_tensor = [ Q.type(torch.DoubleTensor).to(device) for Q in Qs_gt_tensor]

Qs_gt = [Q.cpu().numpy() for Q in Qs_gt_tensor]

adatas = adatas_Qs + [adatas_Rs[-1]]

# Extract cell type labels directly from adatas
ct_labels = [list(adata.obs[key].unique()) for adata in adatas]

Slice pair 4, aligning zf18 to zf24
Slice pair 3, aligning zf12 to zf18
Slice pair 2, aligning zf10 to zf12
Slice pair 1, aligning zf5 to zf10
Slice pair 0, aligning zf3 to zf5


In [11]:
for la in ct_labels:
    print(len(la))

2
9
8
12
14
23


## set ranks

In [12]:
print([Q.shape for Q in Qs_gt])

[(7424, 2), (7136, 9), (8755, 8), (12133, 12), (9903, 14), (14882, 23)]


In [13]:
ranks = [(2,9),(9,8),(8,12),(12,14),(14, 23)]

## load features and spatial coords specific to the triple

In [14]:
s0_name = zf_names[0]
s1_name = zf_names[1]
s2_name = zf_names[2]
s3_name = zf_names[3]
s4_name = zf_names[4]
s5_name = zf_names[5]


filehandle_pair0 = filehandle_zf + 'pair' + str(0) + '/'
filehandle_pair1 = filehandle_zf + 'pair' + str(1) + '/'
filehandle_pair2 = filehandle_zf + 'pair' + str(2) + '/'
filehandle_pair3 = filehandle_zf + 'pair' + str(3) + '/'
filehandle_pair4 = filehandle_zf + 'pair' + str(4) + '/'


X0 = np.load(filehandle_pair0 + s0_name + '_feature.npy')
X1 = np.load(filehandle_pair1 + s1_name + '_feature.npy')
X2 = np.load(filehandle_pair2 + s2_name + '_feature.npy')
X3 = np.load(filehandle_pair3 + s3_name + '_feature.npy')
X4 = np.load(filehandle_pair4 + s4_name + '_feature.npy')
X5 = np.load(filehandle_pair4 + s5_name + '_feature.npy')

Xs = [X0, X1, X2, X3, X4, X5]
Ss = spatial_list

## set OT hyperparameters

In [15]:
gamma=40

# tau_in / epsilon:
tau_in_times_gamma = 5e-6 # as this value -> 0, the in-SR updates become infinitely relaxed. as this value -> +infty, in-SR updates become balanced

tau_out=1e6
tau_in=tau_in_times_gamma / gamma
alpha = 0.0
beta = 0.0

max_iter=75
min_iter=75

new_expr_signal=True

use_bin_annotation=False
n_top_genes=5000 #'all'
n_comps_pca=100
batch_key = 'celltype_new'

print(f"""
Data Processing and HM-OT Parameters:
-----------------------
Gamma: {gamma}
Tau (Out): {tau_out}
Tau (In): {tau_in}
Tau (In) x Gamma: {tau_in_times_gamma}
Alpha: {alpha}
Beta: {beta}
Max Iterations: {max_iter}
Min Iterations: {min_iter}
New Expression Signal: {new_expr_signal}
Using Cell Type Anno: {use_bin_annotation}
Number of Top Genes: {n_top_genes} (if Anno=True, ~5k after DE from Anno)
Number of PCA components: {n_comps_pca}
Batch key: {batch_key}
""")


Data Processing and HM-OT Parameters:
-----------------------
Gamma: 40
Tau (Out): 1000000.0
Tau (In): 1.2500000000000002e-07
Tau (In) x Gamma: 5e-06
Alpha: 0.0
Beta: 0.0
Max Iterations: 75
Min Iterations: 75
New Expression Signal: True
Using Cell Type Anno: False
Number of Top Genes: 5000 (if Anno=True, ~5k after DE from Anno)
Number of PCA components: 100
Batch key: celltype_new



## form cost matrices

In [16]:
importlib.reload(util_zf)

<module 'util_zf' from '/content/drive/Othercomputers/numac/GitHub/WDM/util_zf.py'>

In [17]:
%%time
Aiis_np = []

truncation = 100
time = 50.0
epsilon = 1/gamma

HDM = False
MELD = False

if HDM==False:
    for i, S in enumerate(Ss):
        Aii_np = cdist(S, S)
        Aiis_np += [Aii_np]
elif HDM==True and MELD==False:
    for i, X in enumerate(Xs):
        X_HDM = util_zf.HDM_from_XS(Ss[i], Xs[i], truncation, time)
        Cii_np = cdist(X_HDM, X_HDM)
        Aii_np = np.exp(-Cii_np/epsilon)
        Aiis_np += [Aii_np]
else:
    for i, X in enumerate(Xs):
        X_HDM = util_zf.HDM_from_XS(Xs[i], Xs[i], truncation, time, MELD, epsilon)
        Aii_np = cdist(X_HDM, X_HDM)
        Aiis_np += [Aii_np]


Aiis = [torch.from_numpy(Aii).to(device) for Aii in Aiis_np]

CPU times: user 2.92 s, sys: 916 ms, total: 3.84 s
Wall time: 3.82 s


## factor cost matrices: functions

In [18]:
from util_LR import hadamard_lr, low_rank_distance_factorization, scale_matrix_rows, normalize_mats

def load_data(filehandle_embryo, r=100, r2=15, eps=0.03, device='cpu', \
              feature_handle1 = 'slice1_feature.npy', feature_handle2 = 'slice2_feature.npy',
             spatial_handle1 = 'slice1_coordinates.npy', spatial_handle2 = 'slice2_coordinates.npy',  hadamard = True):

    data_t1 = np.load(filehandle_embryo + feature_handle1)
    data_t2 = np.load(filehandle_embryo + feature_handle2)

    n, m = data_t1.shape[0], data_t2.shape[0]
    data_t1 = scale_matrix_rows(data_t1)
    data_t2 = scale_matrix_rows(data_t2)

    data_t1 = torch.from_numpy(data_t1).to(device)
    data_t2 = torch.from_numpy(data_t2).to(device)

    spatial_t1 =  torch.from_numpy(np.load(filehandle_embryo + spatial_handle1)).to(device)
    spatial_t2 =  torch.from_numpy(np.load(filehandle_embryo + spatial_handle2)).to(device)

    # Factorize C
    V_C, U_C = low_rank_distance_factorization(data_t1, data_t2, r, eps, device=device)

    n, m = V_C.shape[0], U_C.shape[1]

    # M1 matrix
    V_A, U_A = low_rank_distance_factorization(data_t1, data_t1, r2, eps, device=device)
    V_B, U_B = low_rank_distance_factorization(spatial_t1, spatial_t1, r2, eps, device=device)

    print(f'shape of V_C: {V_C.shape}')
    print(f'shape of U_C: {U_C.shape}')
    print(f'shape of V_A: {V_A.shape}')
    print(f'shape of U_A: {U_A.shape}')
    print(f'shape of V_B: {V_B.shape}')
    print(f'shape of U_B: {U_B.shape}')

    return (V_C, U_C), (V_A, U_A), (V_B, U_B)

    if hadamard:
        M1_A, M1_B = hadamard_lr(V_A, U_A.T, V_B, U_B.T, device=device)
    else:
        # Default to spatial dist
        M1_A, M1_B = V_B, U_B.T

    # M2 matrix
    V_A, U_A = low_rank_distance_factorization(data_t2, data_t2, r2, eps, device=device)
    V_B, U_B = low_rank_distance_factorization(spatial_t2, spatial_t2, r2, eps, device=device)

    if hadamard:
        M2_A, M2_B = hadamard_lr(V_A, U_A.T, V_B, U_B.T, device=device)
    else:
        # Default to spatial dist
        M2_A, M2_B = V_B, U_B.T

    del V_A, U_A, V_B, U_B

    #V_C, U_C, M1_A, M1_B, M2_A, M2_B = normalize_mats(V_C, U_C, M1_A, M1_B, M2_A, M2_B)



In [19]:
import random

def estimate_max_norm(A, B):
    assert A.shape[1] == B.shape[0], 'Inner matrix dimensions must equal.'
    col_norms = torch.linalg.norm(A, axis=0)
    row_norms = torch.linalg.norm(B, axis=1)
    colrow_prods = col_norms*row_norms
    C_max = torch.max(colrow_prods)
    return C_max

def normalize_mats(V_C, U_C, M1_A, M1_B, M2_A, M2_B, device='cpu'):
    n = V_C.shape[0]

    norm_constant1 = estimate_max_norm(V_C, U_C)**1/2
    norm_constant2 = estimate_max_norm(M1_A, M1_B)**1/2
    norm_constant3 = estimate_max_norm(M2_A, M2_B)**1/2

    V_C, U_C = V_C/norm_constant1, U_C/norm_constant1
    M1_A, M1_B = M1_A/norm_constant2, M1_B/norm_constant2
    M2_A, M2_B = M2_A/norm_constant2, M2_B/norm_constant3
    '''
    Q,R,T, errs = FRLC_LRDist.FRLC_LR_opt((V_C, U_C), (M1_A, M1_B), (M2_A, M2_B), r=2, max_iter=2, \
                                          min_iter=2, printCost=True, device=device, returnFull=False)
    del Q,R,T
    cGW = errs['GW_cost'][0]
    cW = errs['W_cost'][0]
    '''
    c = (norm_constant1 / norm_constant2**2)**1/2
    V_C, U_C = V_C*c, U_C*c

    return V_C, U_C, M1_A, M1_B, M2_A, M2_B

def low_rank_distance_factorization(X1, X2, r, eps, device='cpu', dtype=torch.float64):
    n = X1.shape[0]
    m = X2.shape[0]
    '''
    Indyk '19
    '''
    # low-rank distance matrix factorization of Bauscke, Indyk, Woodruff

    t = int(r/eps) # this is poly(1/eps, r) in general -- this t might not achieve the correct bound tightly
    i_star = random.randint(1, n)
    j_star = random.randint(1, m)

    # Define probabilities of sampling
    p = (torch.cdist(X1, X2[j_star][None,:])**2 \
            + torch.cdist(X1[i_star,:][None,:], X2[j_star,:][None,:])**2 \
                    + (torch.sum(torch.cdist(X1[i_star][None,:], X2))/m) )[:,0]**2

    p_dist = (p / p.sum())

    # Use random choice to sample rows
    indices_p = torch.from_numpy(np.random.choice(n, size=(t), p=p_dist.cpu().numpy())).to(device)
    X1_t = X1[indices_p, :]
    '''
    Frieze '04
    '''
    P_t = torch.sqrt(p[indices_p]*t)
    S = torch.cdist(X1_t, X2)/P_t[:, None] # t x m

    # Define probabilities of sampling by row norms
    q = torch.norm(S, dim=0)**2 / torch.norm(S)**2 # m x 1
    q_dist = (q / q.sum())
    # Use random choice to sample rows
    indices_q = torch.from_numpy(np.random.choice(m, size=(t), p=q_dist.cpu().numpy())).to(device)
    S_t = S[:, indices_q] # t x t
    Q_t = torch.sqrt(q[indices_q]*t)
    W = S_t[:, :] / Q_t[None, :]
    # Find U
    U, Sig, Vh = torch.linalg.svd(W) # t x t for all
    F = U[:, :r] # t x r
    # U.T for the final return
    U_t = (S.T @ F) / torch.norm(W.T @ F) # m x r
    '''
    Chen & Price '17
    '''
    # Find V for the final return
    indices = torch.from_numpy(np.random.choice(m, size=(t))).to(device)
    X2_t = X2[indices, :] # t x dim
    D_t = torch.cdist(X1, X2_t) / np.sqrt(t) # n x t
    Q = U_t.T @ U_t # r x r
    U, Sig, Vh = torch.linalg.svd(Q)
    U = U / Sig # r x r
    U_tSub = U_t[indices, :].T # t x r
    B = U.T @ U_tSub / np.sqrt(t) # (r x r) (r x t)
    A = torch.linalg.inv(B @ B.T)
    Z = ((A @ B) @ D_t.T) # (r x r) (r x t) (t x n)
    V = Z.T @ U
    return V.double(), U_t.T.double()

## factor cost matrices

In [35]:
importlib.reload(util_zf)

<module 'util_zf' from '/content/drive/Othercomputers/numac/GitHub/WDM/util_zf.py'>

In [36]:
%%time

z = 50
#c = 1
#eps=0.02

if new_expr_signal==True:

    C_factors_sequence = []
    A_factors_sequence = []

    for i in range(len(ranks)):

        s1_name = zf_names[i]
        s2_name = zf_names[i+1]

        print(f'Loading slices {s1_name} to {s2_name}')

        r1, r2 = ranks[i]
        nidx_1, nidx_2 = exclude_rows[i], exclude_rows[i+1]

        filehandle_pair = filehandle_zf + 'pair' + str(i) + '/'
        save_dir = filehandle_pair
        C_matrix = cdist(Xs[i], Xs[i+1])


        C_factors12, A_factors11, A_factors22 = util_zf.factor_mats_tens(torch.Tensor(C_matrix).to(device), Aiis[i], Aiis[i+1], \
                                                        device, z=50, c=100)

        C_factors_sequence.append(C_factors12)
        if i == range(len(ranks))[-1]:
            A_factors_sequence.append(A_factors11)
            A_factors_sequence.append(A_factors22)
        else:
            A_factors_sequence.append(A_factors11)

    c = max([torch.max(C[0] @ C[1]) for C in C_factors_sequence])
    c = max(c, 1/c)

    for i in range(len(C_factors_sequence)):
        C = C_factors_sequence[i]
        C_factors_sequence[i] = (C[0]/c, C[1]/c)
else:
    pass


Loading slices zf3 to zf5
Matrix shapes for debugging: torch.Size([7424, 50]), torch.Size([50, 7136])
Matrix shapes for debugging: torch.Size([7424, 50]), torch.Size([7424, 50]), torch.Size([50, 7136]), torch.Size([50, 7136])
C done
Loading slices zf5 to zf10
Matrix shapes for debugging: torch.Size([7136, 50]), torch.Size([50, 7136])
Matrix shapes for debugging: torch.Size([7136, 50]), torch.Size([7136, 50]), torch.Size([50, 8755]), torch.Size([50, 8755])
C done
Loading slices zf10 to zf12
Matrix shapes for debugging: torch.Size([8755, 50]), torch.Size([50, 8755])
Matrix shapes for debugging: torch.Size([8755, 50]), torch.Size([8755, 50]), torch.Size([50, 12133]), torch.Size([50, 12133])
C done
Loading slices zf12 to zf18
Matrix shapes for debugging: torch.Size([12133, 50]), torch.Size([50, 9903])
Matrix shapes for debugging: torch.Size([12133, 50]), torch.Size([12133, 50]), torch.Size([50, 9903]), torch.Size([50, 9903])
C done
Loading slices zf18 to zf24
Matrix shapes for debugging: t

## save factors

In [37]:
import os

save_dir = filehandle_save_factors

if new_expr_signal == True:
    os.makedirs(save_dir, exist_ok=True)
    N = len(A_factors_sequence)
    if len(C_factors_sequence) != N - 1:
        raise ValueError("C_factors_sequence must be of length N - 1, where N is the length of A_factors_sequence.")


    for idx in range(N):
        A_tuple = A_factors_sequence[idx]
        for arr_idx, array in enumerate(A_tuple):
            array_np = array.cpu().numpy()
            if not isinstance(array_np, np.ndarray):
                raise TypeError(f"Element at A_factors_sequence[{idx}][{arr_idx}] is not a NumPy array.")
            filename = os.path.join(save_dir, f'A_array_{idx}_{arr_idx}.npy')
            try:
                np.save(filename, array_np)
                print(f"Saved {filename}")
            except Exception as e:
                print(f"Failed to save {filename}: {e}")

        if idx < N - 1:
            C_tuple = C_factors_sequence[idx]
            for arr_idx, array in enumerate(C_tuple):
                array_np = array.cpu().numpy()
                if not isinstance(array_np, np.ndarray):
                    raise TypeError(f"Element at C_factors_sequence[{idx}][{arr_idx}] is not a NumPy array.")
                filename = os.path.join(save_dir, f'C_array_{idx}_{arr_idx}.npy')
                try:
                    np.save(filename, array_np)
                    print(f"Saved {filename}")
                except Exception as e:
                    print(f"Failed to save {filename}: {e}")
else:
    print("new_expr_signal is False; skipping save.")

Saved drive/Othercomputers/numac/GitHub/celltypediscovery/_wdm_save_sc_factors/A_array_0_0.npy
Saved drive/Othercomputers/numac/GitHub/celltypediscovery/_wdm_save_sc_factors/A_array_0_1.npy
Saved drive/Othercomputers/numac/GitHub/celltypediscovery/_wdm_save_sc_factors/C_array_0_0.npy
Saved drive/Othercomputers/numac/GitHub/celltypediscovery/_wdm_save_sc_factors/C_array_0_1.npy
Saved drive/Othercomputers/numac/GitHub/celltypediscovery/_wdm_save_sc_factors/A_array_1_0.npy
Saved drive/Othercomputers/numac/GitHub/celltypediscovery/_wdm_save_sc_factors/A_array_1_1.npy
Saved drive/Othercomputers/numac/GitHub/celltypediscovery/_wdm_save_sc_factors/C_array_1_0.npy
Saved drive/Othercomputers/numac/GitHub/celltypediscovery/_wdm_save_sc_factors/C_array_1_1.npy
Saved drive/Othercomputers/numac/GitHub/celltypediscovery/_wdm_save_sc_factors/A_array_2_0.npy
Saved drive/Othercomputers/numac/GitHub/celltypediscovery/_wdm_save_sc_factors/A_array_2_1.npy
Saved drive/Othercomputers/numac/GitHub/celltypedi

## load factors

In [38]:
# Get all files in the directory
saved_files = os.listdir(save_dir)

# Filter files for C and A arrays
C_files = [f for f in saved_files if f.startswith('C_array_') and f.endswith('.npy')]
A_files = [f for f in saved_files if f.startswith('A_array_') and f.endswith('.npy')]

def parse_filename(filename, prefix):
    """Extracts idx and arr_idx from the filename."""
    # Remove prefix and suffix
    s = filename[len(prefix):-4]  # Remove prefix and '.npy'
    idx_str, arr_idx_str = s.split('_')
    idx = int(idx_str)
    arr_idx = int(arr_idx_str)
    return idx, arr_idx

# Build dictionaries to organize the files by indices
A_dict = {}
for f in A_files:
    idx, arr_idx = parse_filename(f, 'A_array_')
    if idx not in A_dict:
        A_dict[idx] = {}
    A_dict[idx][arr_idx] = f

C_dict = {}
for f in C_files:
    idx, arr_idx = parse_filename(f, 'C_array_')
    if idx not in C_dict:
        C_dict[idx] = {}
    C_dict[idx][arr_idx] = f

# Reconstruct A_factors_sequence_test
A_factors_sequence_test = []
for idx in sorted(A_dict.keys()):
    arr_dict = A_dict[idx]
    arr_list = []
    for arr_idx in sorted(arr_dict.keys()):
        filename = arr_dict[arr_idx]
        array = torch.Tensor(np.load(os.path.join(save_dir, filename))).type(torch.DoubleTensor).to(device)
        arr_list.append(array)
    A_factors_sequence_test.append(tuple(arr_list))

# Reconstruct C_factors_sequence_test
C_factors_sequence_test = []
for idx in sorted(C_dict.keys()):
    arr_dict = C_dict[idx]
    arr_list = []
    for arr_idx in sorted(arr_dict.keys()):
        filename = arr_dict[arr_idx]
        array = torch.Tensor(np.load(os.path.join(save_dir, filename))).type(torch.DoubleTensor).to(device)
        arr_list.append(array)
    C_factors_sequence_test.append(tuple(arr_list))

# Optional: Print the reconstructed sequences to verify
print("Reconstructed C_factors_sequence_test:")
for idx, tup in enumerate(C_factors_sequence_test):
    print(f"Index {idx}: Tuple of length {len(tup)}")

print("\nReconstructed A_factors_sequence_test:")
for idx, tup in enumerate(A_factors_sequence_test):
    print(f"Index {idx}: Tuple of length {len(tup)}")

Reconstructed C_factors_sequence_test:
Index 0: Tuple of length 2
Index 1: Tuple of length 2
Index 2: Tuple of length 2
Index 3: Tuple of length 2
Index 4: Tuple of length 2

Reconstructed A_factors_sequence_test:
Index 0: Tuple of length 2
Index 1: Tuple of length 2
Index 2: Tuple of length 2
Index 3: Tuple of length 2
Index 4: Tuple of length 2
Index 5: Tuple of length 2


In [39]:
A_factors_sequence = A_factors_sequence_test
C_factors_sequence = C_factors_sequence_test

## run `WDM`

In [40]:
%%time
importlib.reload(WassersteinDiffMap)
importlib.reload(WassersteinDiffMapScaling)


WDM = WassersteinDiffMap.WassersteinDifferentiationMapping(ranks, tau_in = tau_in, tau_out=tau_out, \
                      gamma=gamma, max_iter=max_iter, min_iter=min_iter, device=device, dtype=torch.float64, \
                     printCost=True, returnFull=False, alpha=alpha, beta=beta, \
                      initialization='Full', init_args = None)

WDM.gamma_smoothing(C_factors_sequence, A_factors_sequence)

Qs_pred = [Q.cpu().numpy() for Q in WDM.Q_gammas]
Ts_pred = [T.cpu().numpy() for T in WDM.T_gammas]

Iteration: 0


RuntimeError: shape '[7424, 2500]' is invalid for input of size 2755788800

## plot

In [26]:
importlib.reload(clustering)

<module 'clustering' from '/content/drive/Othercomputers/numac/GitHub/WDM/clustering.py'>

In [27]:
Ss = spatial_list

node_labels1 = [None for Q in WDM.Q_gammas]

clustering.plot_clusters_from_QT(Ss, Qs_pred, Ts_pred, node_labels1, dotsize=10)
clustering.diffmap_from_QT_sankey(Qs_pred, Ts_pred, cell_type_labels=node_labels1, dsf=.3, plot_height=1500,plot_width=3000, save_name='diffmap_pred')
clustering.diffmap_from_QT(Qs_pred, Ts_pred, cell_type_labels=node_labels1, dsf=.3, stretch=1.5)

NameError: name 'Qs_pred' is not defined

In [ ]:
importlib.reload(clustering)

## save

In [ ]:
for i,Q in enumerate(WDM.Q_gammas):
    np.save(filehandle_save + f'wdm_Q{i}.npy', Q.cpu().numpy())

for i,T in enumerate(WDM.T_gammas):
    np.save(filehandle_save + f'wdm_T{i}{i+1}.npy', T.cpu().numpy())

## ari, ami

In [ ]:
importlib.reload(util_zf)

In [ ]:
gt_types_list = clustering.max_likelihood_clustering(Qs_gt)
pred_types_list = clustering.max_likelihood_clustering(Qs_pred)

util_zf.compute_ARI_and_AMI(gt_types_list, pred_types_list)

In [ ]:
importlib.reload(util_zf)

In [ ]:
# util_zf.silhouette(gt_types_list, pred_types_list, Xs, Ss)

In [ ]:
# util_zf.cos_silhouette(gt_types_list, pred_types_list, Xs, Ss)

## freezing $Q$s with annotations, running `WDM`

In [ ]:
%%time
Q_annot = Qs_gt_tensor

importlib.reload(WassersteinDiffMap)

WDM2 = WassersteinDiffMap.WassersteinDifferentiationMapping(ranks, tau_in = tau_in, tau_out=tau_out, \
                      gamma=gamma, max_iter=max_iter, min_iter=min_iter, device=device, dtype=torch.float64, \
                     printCost=True, returnFull=False, alpha=alpha, beta=beta, \
                      initialization='Full', init_args=None)

WDM2.impute_annotated_transitions(C_factors_sequence, A_factors_sequence, Q_annot)
T_gammas = WDM2.T_gammas

In [ ]:
for i,Q in enumerate(WDM2.Q_gammas):
    np.save(filehandle_save + f'ann_Q{i}.npy', Q.cpu().numpy())

for i,T in enumerate(WDM2.T_gammas):
    np.save(filehandle_save + f'ann_T{i}{i+1}.npy', T.cpu().numpy())

# Analysis part 1: entropy of transitions

In [ ]:
Ts_ann = T_gammas
Ts_pred = Ts_pred

## entropy functions

In [ ]:
importlib.reload(util_zf)

In [ ]:
print('Comparing entropies with annotations\n')

util_zf.compare_T_entropies(Ts_ann, Ts_pred)
print('\n')
util_zf.compare_T_col_entropies(Ts_ann, Ts_pred)
print('\n')
util_zf.compare_T_row_entropies(Ts_ann, Ts_pred)

## convergence / collision

In [ ]:
Ts_moscot = []

for i in range(5):
    Pi = np.load(filehandle_load_moscot + f'moscot_Pi_{i}.npy')
    Q1 = Qs_gt[i]
    Q2 = Qs_gt[i+1]
    T = ((Q1.T @ Pi) @ Q2) * (Q1.shape[0]*Q2.shape[0])
    Ts_moscot += [T]

In [ ]:
importlib.reload(util_zf)

In [ ]:
print('Comparing entropies with moscot\n')

Ts_moscot_tens = [ torch.Tensor(T) for T in Ts_moscot ]

util_zf.compare_T_entropies(Ts_moscot_tens, Ts_pred)
print('\n')
util_zf.compare_T_col_entropies(Ts_moscot_tens, Ts_pred)
print('\n')
util_zf.compare_T_row_entropies(Ts_moscot_tens, Ts_pred)

In [ ]:
importlib.reload(util_zf)

'''
for i, (T_pred, T_ann, T_moscot) in enumerate(zip(Ts_pred, Ts_ann, Ts_moscot)):
    T_ann = T_ann.cpu().numpy()
    util_zf.plot_collision_profiles([T_pred,
                                     T_ann,
                                     T_moscot],
                                    title=f'Collision / convergence profiles at snapshot {i+2}',
                                    label_list=['HM-OT', 'Anno', 'moscot'])
''';

## anno diffmap

In [ ]:
importlib.reload(clustering)

In [ ]:
clustering.plot_clusters_from_QT(Ss, Qs_gt, Ts_ann, None, dotsize=10)
clustering.diffmap_from_QT_sankey(Qs_gt, [T.cpu().numpy() for T in Ts_ann], cell_type_labels=ct_labels, dsf=.3, plot_height=1500, plot_width=3000, save_name='diffmap_anno')
clustering.diffmap_from_QT(Qs_gt, Ts_ann, cell_type_labels=ct_labels, dsf=.3, stretch=1.5)

In [ ]:
clustering.diffmap_from_QT_sankey(Qs_gt, Ts_moscot, cell_type_labels=ct_labels, dsf=.3, plot_height=1500, plot_width=3000, save_name='diffmap_wot')
clustering.diffmap_from_QT(Qs_gt, Ts_moscot, cell_type_labels=ct_labels, dsf=.3, stretch=1.5)


## independence ratio

In [ ]:
def independence_ratio(T12, T23, g_Q2, g_Q1, g_Q3, labels1, labels3, diagonal=False):
    ct_to_IR = { }
    set1, set3 = set(labels1), set(labels3)
    labels_intersection = set1.intersection(set3)
    for label in labels_intersection:
        idx_k = labels1.index(label)
        idx_l = labels3.index(label)
        #print(f'{label}: {idx_k} to {idx_l}')
        vec = (T12[idx_k, :] * T23[:, idx_l]) / g_Q2
        ct_to_IR[label] = torch.sum(vec) / (g_Q1[idx_k] * g_Q3[idx_l])
    return ct_to_IR

## Peter's desired trajectories + UMAP plot

In [ ]:
ranks

In [ ]:
sizes = [2, 9, 8, 12, 14, 23]

In [ ]:
import numpy as np

def normalize_to_row_stochastic(Ts):
    """
    Normalize a list of coupling matrices to be row-stochastic (i.e., each row sums to 1).

    Args:
        Ts (list of np.ndarray): A list of coupling matrices (2D numpy arrays).

    Returns:
        normalized_Ts (list of np.ndarray): A list of normalized row-stochastic coupling matrices.
    """
    normalized_Ts = []

    first_marginal = Ts[0].sum(axis=1, keepdims=False)

    for T in Ts:
        # Normalize each row of the matrix to sum to 1
        row_sums = T.sum(axis=1, keepdims=False)
        print(f'row_sums shape: {row_sums.shape}')
        print(f'T shape: {T.shape}')
        # Handle any rows that sum to zero by avoiding division by zero
        normalized_T = np.diag(1/row_sums) @ T
        normalized_Ts.append(normalized_T)

    return first_marginal, normalized_Ts


# for now, sample according to Ts, in future we can specify an initial cluster or even initial point
def sample_latent_trajectories(Ts_rs, g_init, sizes, num_trajectories=10):
    """
    Sample trajectories from the given latent coupling matrices Ts.

    Args:
        Ts_rs (list of np.ndarray): A list of coupling matrices T_t for t = 1, ..., N-1.
                                    These should be row-stochastic, but we ensure they are normalized.
        g_init (np.ndarray): The initial distribution over states.
        sizes (list of int): The sizes of the state spaces at each timepoint.
        num_trajectories (int): Number of trajectories to sample.

    Returns:
        trajectories (np.ndarray): An array of sampled trajectories of shape (num_trajectories, N).
    """
    N = len(sizes)  # Number of timepoints
    trajectories = np.zeros((num_trajectories, N), dtype=int)

    # Normalize initial distribution
    g_init = g_init / np.sum(g_init)

    # Sample initial state
    trajectories[:, 0] = np.random.choice(sizes[0], num_trajectories, p=g_init)

    # Sample subsequent states
    for t in range(1, N):
        for i in range(num_trajectories):
            prev_state = trajectories[i, t-1]
            # Normalize the row of Ts_rs corresponding to the previous state
            row = Ts_rs[t-1][prev_state]
            row = row / np.sum(row)  # Normalize to ensure it sums to 1
            # Sample next state based on the normalized row
            trajectories[i, t] = np.random.choice(sizes[t], p=row)

    return trajectories

In [ ]:
g_init, Ts_rs = normalize_to_row_stochastic(Ts_pred)

g_init

traj_test = sample_latent_trajectories(Ts_rs=Ts_rs, g_init=g_init, sizes=sizes)
traj_test

## what about arg-secondmax? Does this have biological signal?

In [ ]:
labels_2nd = []

for Q in Qs_pred:
    # Get the indices of the sorted entries for each row
    sorted_indices = np.argsort(Q, axis=1)

    # Extract the second-to-last index (the second largest) from the sorted indices
    second_largest_indices = sorted_indices[:, -2]
    labels_2nd += [second_largest_indices]

In [ ]:
threshold = 0.1  # Example threshold as a percentage of the largest value

labels_consensus = []

for Q in Qs_pred:
    argmax_labels = np.argmax(Q, axis=1)
    sorted_indices = np.argsort(Q, axis=1)
    second_argmax_labels = sorted_indices[:, -2]

    # Get the largest and second largest values from Q
    largest_values = np.max(Q, axis=1)
    second_largest_values = Q[np.arange(Q.shape[0]), second_argmax_labels]

    # Apply the threshold to choose between argmax and second argmax
    consensus_labels = np.where((largest_values - second_largest_values) / largest_values < threshold,
                                second_argmax_labels, argmax_labels)

    # Convert consensus_labels (NumPy array) to a regular Python list of ints
    labels = consensus_labels.tolist()  # Convert the array to a list
    labels = [int(label) for label in labels]  # Ensure all elements are ints
    labels = np.array(labels)
    labels_consensus.append(labels)  # Use append to add the new list of labels

# Now labels_consensus is a list of lists, suitable for plotting
clustering.plot_clustering_list(Ss, labels_consensus, dotsize=10)

In [ ]:
util_zf.compute_ARI_and_AMI(gt_types_list, labels_consensus)